Import the libraries

In [84]:
import numpy as np
import pandas as pd 

# For portfolio optimization
from scipy.optimize import minimize 
import plotly.graph_objs as go

Reading Finance Data

In [85]:
# import the data from  local file
stock_data=pd.read_excel('Daily_Returns.xlsx')

# Convert the 'date' column to datetime format 
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

# Set the 'date' column as the index
stock_data.set_index('Date', inplace=True)

IMP Functions for Portfolio Construction

In [86]:
# get the names of the stocks
def get_stock_names(df):
    stock_names = df.columns[0:].tolist()
    return stock_names

names = get_stock_names(stock_data)

In [87]:
# getting covarince matrix for portfolios

def get_cov_matrix(df):
    # Calculate the covariance matrix for each stock
    cov_matrix = df.cov()
    
    return cov_matrix

# calling the function
cov_matrix = get_cov_matrix(stock_data)

In [88]:
# getting ER of the portfolio

def get_expected_returns(df):
    
    # Calculate the mean return for each stock
    expected_returns = df.mean()
    
    return expected_returns

expected_returns = get_expected_returns(stock_data)

In [89]:
#getting the correlation matrix
def get_cor_matrix(df):
    # Calculate the correlation matrix for each stock
    cor_matrix = df.corr()
    
    return cor_matrix

cor_matrix=get_cor_matrix(stock_data)


Optimally Weighted Portfolios

- To generate optimally weighted portfolis, we use `Mean Variance Optimization`.
- The `SLSQP` algorithm (Sequential Least Squares Programming) allows us to minimize a function with several variables and constraints.
- We are minimizing the negative expected return s.t. 3 constraints are satisfied;
    - Sum of weights should be one
    - Weights should not be negative
    - Risk should be below some tolerance level

In [90]:
number_of_stocks = len(names)

# generatig weights
initial_weights = [1/number_of_stocks] * number_of_stocks

constraints = ({'type': 'eq', 'fun': lambda wt: np.sum(wt) - 1}, # sum of weights to be 1
               {'type': 'ineq', 'fun': lambda wt: wt}, # weight should be positive
               {'type': 'ineq', 'fun': lambda wt: 0.1 - np.sqrt(np.dot(wt, np.dot(cov_matrix, wt.T)))}) # risk constraint

algorithm = 'SLSQP'  #Sequential Least Squares Programming

# defining objective to optimise
def objective(wt, expected_returns):
    return -np.sum(wt * expected_returns)  # maximize return

# generating weights
def get_weights(expected_returns, cov_matrix, initial_weights=initial_weights, 
                algorithm=algorithm,constraints=constraints):
    
    result = minimize(objective,
                        initial_weights,
                        args=(expected_returns),
                        method=algorithm,
                        constraints=constraints)
    
    return result

In [91]:
# calling the function
weights= get_weights(expected_returns,cov_matrix)

Checking the risk constraint

In [93]:
Risk = np.sqrt(np.dot(weights.x,np.dot(weights.x,cov_matrix.to_numpy())))
print(Risk)

0.018169128037068275


In [95]:
# map weights to stock names

def map_weights_to_names(optimized_weights, names):
    weights_dict = dict(zip(names, optimized_weights))
    
    # Filter out stocks with weights greater than 0
    selected_stocks = {stock: weight for stock, weight in weights_dict.items() if weight > 0}
    
    return selected_stocks

# function call
optimized_weights = weights.x

selected_stocks = map_weights_to_names(optimized_weights, names)

Checking whether weights sum upto 1

In [94]:
# Calculate the sum of all weights
total_weight = sum(weights.x)

print("Total weight:", total_weight)

Total weight: 0.9999999999999993


Looping it all